In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
klin = np.loadtxt('../flowpm/data/Planck15_a1p00.txt').T[0]
plin = np.loadtxt('../flowpm/data/Planck15_a1p00.txt').T[1]
stages = np.linspace(0.1, 1.0, 5, endpoint=True)

In [4]:
from scipy.interpolate import InterpolatedUnivariateSpline as iuspline

ipklin = iuspline(klin, plin)

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import mesh_tensorflow as mtf

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
# Define the cluster
# cluster = tf.train.ClusterSpec({"mesh": ["localhost:2222", "localhost:2223"]})

# import flowpm.mesh_ops as mpm

nblockx = 2
nblocky = 2
nproc = nblockx*nblocky

mesh_hosts = ["localhost:%d"%(8222+j) for j in range(nproc)]

# Create a cluster from the mesh hosts.                                                                                                                                                                                                                                                                                                                                  
cluster = tf.train.ClusterSpec({"mesh": mesh_hosts, "master":["localhost:8488"]})

# Create a server for local mesh members                                                                                                                                                                                                                                                                                                                                 
server = tf.train.Server(cluster,
                       job_name="master",
                       task_index=0)

devices = ['/job:mesh/task:%d'%i for i in range(cluster.num_tasks("mesh"))]


In [7]:
import sys
sys.path.pop(6)
sys.path.append('../')
sys.path.append('../flowpm/')


In [8]:
# import flowpm.mesh_ops as mpm
import flowpm
from flowpm import mesh_ops
from flowpm import mesh_utils
from flowpm import mesh_kernels

import flowpm.mesh_utils as mpu
import flowpm.mtfpm as fpm
import tensorflow_probability as tfp


graph = mtf.Graph()
mesh = mtf.Mesh(graph, "my_mesh")

batch_size = 1
boxsize=100
nc = 64       # Native resolution of the grid

# Parameters of the small scales decomposition
n_block_x = nblockx
n_block_y = nblocky
n_block_z = 1
halo_size = 8

# Parameters of the large scales decomposition
downsampling_factor = 2
lnc = nc // 2**downsampling_factor 


# Dimensions of the low resolution grid
x_dim = mtf.Dimension("nx_lr", lnc)
y_dim = mtf.Dimension("ny_lr", lnc)
z_dim = mtf.Dimension("nz_lr", lnc)

nx_dim = mtf.Dimension('nx_block', n_block_x)
ny_dim = mtf.Dimension('ny_block', n_block_y)
nz_dim = mtf.Dimension('nz_block', n_block_z)

sx_dim = mtf.Dimension('sx_block', nc//n_block_x)
sy_dim = mtf.Dimension('sy_block', nc//n_block_y)
sz_dim = mtf.Dimension('sz_block', nc//n_block_z)

lx_dim = mtf.Dimension('lx_block', lnc//n_block_x)
ly_dim = mtf.Dimension('ly_block', lnc//n_block_y)
lz_dim = mtf.Dimension('lz_block', lnc//n_block_z)

batch_dim = mtf.Dimension("batch", batch_size)
pk_dim = mtf.Dimension("npk", len(plin))
pk = mtf.import_tf_tensor(mesh, plin.astype('float32'), shape=[pk_dim])

In [9]:
hr_shape = [batch_dim, 
            nx_dim, ny_dim, nz_dim,
            sx_dim, sy_dim, sz_dim]

#trying reshapes with this
lr_shape2 = [batch_dim, 
            nx_dim, 
            lx_dim, y_dim, z_dim]

lr_shape = [batch_dim, 
            x_dim, y_dim, z_dim]

hr_shape, lr_shape


([Dimension(name='batch', size=1),
  Dimension(name='nx_block', size=2),
  Dimension(name='ny_block', size=2),
  Dimension(name='nz_block', size=1),
  Dimension(name='sx_block', size=32),
  Dimension(name='sy_block', size=32),
  Dimension(name='sz_block', size=64)],
 [Dimension(name='batch', size=1),
  Dimension(name='nx_lr', size=16),
  Dimension(name='ny_lr', size=16),
  Dimension(name='nz_lr', size=16)])

In [10]:

initial_conditions = flowpm.linear_field(nc,          # size of the cube
                                         boxsize,     # Physical size of the cube
                                         ipklin,      # Initial powerspectrum
                                         batch_size=batch_size)
initc = tf.reshape(initial_conditions, [1, nblockx, nblocky, 1, nc//nblockx, nc//nblocky, nc])

field = mtf.import_tf_tensor(mesh, initc, shape=hr_shape)


#commented code not needed to replicate example
# for block_size_dim in hr_shape[-3:]:
#     field = mtf.pad(field, [halo_size, halo_size], block_size_dim.name)

# for blocks_dim, block_size_dim in zip(hr_shape[1:4], field.shape[-3:]):
#     field = mesh_ops.halo_reduce(field, blocks_dim, block_size_dim, halo_size)

field = mtf.reshape(field, field.shape+[mtf.Dimension('h_dim', 1)])
high = field
low = mesh_utils.downsample(field, downsampling_factor, antialias=True)

low = mtf.reshape(low, low.shape[:-1])
high = mtf.reshape(high, high.shape[:-1])
print(high)
print(low)
#commented code not needed to replicate example
# for block_size_dim in hr_shape[-3:]:
#     low = mtf.slice(low, halo_size//2**downsampling_factor, block_size_dim.size//2**downsampling_factor, block_size_dim.name)

low = mtf.reshape(low, lr_shape)

#Trying random stuff here
# low = mtf.reshape(low, low.shape+[mtf.Dimension('l_dim', 1)])
# lshape = low.shape
# print(low)
# low = mtf.transpose(low, new_shape=lshape[:2]+[lshape[-1]]+lshape[3:-1]+[lshape[2]])
# # low = mtf.reshape(low, low.shape[:2]+low.shape[3:5] + lr_shape[2:])
# low = mtf.reshape(low, low.shape[:2]+low.shape[3:5] + lr_shape[2:])

low

Tensor[reshape_2:0, Shape[batch=1, nx_block=2, ny_block=2, nz_block=1, sx_block=32, sy_block=32, sz_block=64], <dtype: 'float32'>]
Tensor[reshape_1:0, Shape[batch=1, nx_block=2, ny_block=2, nz_block=1, sx_block=8, sy_block=8, sz_block=16], <dtype: 'float32'>]


Tensor[reshape_3:0, Shape[batch=1, nx_lr=16, ny_lr=16, nz_lr=16], <dtype: 'float32'>]

In [11]:
# Let's have a look!
devices = ["/job:mesh/task:%d"%i for i in range(cluster.num_tasks("mesh"))]
mesh_shape = [("row", nblockx), ("col", nblocky)]
layout_rules = [("nx_lr", "row"), ("ny_lr", "col"),
                ("nx", "row"), ("ny", "col"),
                ("nx_block","row"), ("ny_block","col"),
               ("lx_block","row"), ("ly_block","col")]

mesh_impl = mtf.placement_mesh_impl.PlacementMeshImpl(
    mesh_shape, layout_rules, devices)

lowering = mtf.Lowering(graph, {mesh:mesh_impl})

tf_field = lowering.export_to_tf_tensor(low)

NotImplementedError: Try first reshaping to insert a new tf dimension, then changing layout. input_shape=Shape[batch=1, nx_block=2, ny_block=2, nz_block=1, sx_block=8, sy_block=8, sz_block=16] output_shape=Shape[batch=1, nx_lr=16, ny_lr=16, nz_lr=16]

In [24]:
with tf.Session("grpc://localhost:8488") as sess:
    fin_ref = sess.run(tf_field )

In [25]:
fin_ref.shape

(1, 2, 1, 1, 8, 8, 16, 2)